In [12]:
import logging

from pytezos.sandbox.node import SandboxedNodeTestCase
from pytezos.sandbox.parameters import sandbox_addresses, sandbox_commitment
from pytezos import ContractInterface, pytezos, MichelsonRuntimeError
from pytezos.contract.result import ContractCallResult

import unittest
import time

In [13]:
contract_fn = '../build/tz/baking_bet.tz'

RUN_TIME = 0
ONE_HOUR = 60*60
ONE_DAY = ONE_HOUR*24

In [14]:
class ContractInteractionsTestCase(SandboxedNodeTestCase):

    def test_contract_interactions(self):

        # four participants and their pk hashes:
        self.a = 'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ'
        self.b = 'tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos'
        self.c = 'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'
        self.d = 'tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM'

        self.manager = self.a

        self.oracle_address = 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn'
        # florencenet: KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn
        # edo2net:     KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ

        self.currency_pair = 'XTZ-USD'
        self.measure_start_fee = 200_000
        self.expiration_fee = 100_000
        self.current_time = RUN_TIME

        # this is eventId that for the tests:
        self.id = 0

        self.default_event_params = {
            'currencyPair': self.currency_pair,
            'targetDynamics': 1_000_000,
            'betsCloseTime': RUN_TIME + 24*ONE_HOUR,
            'measurePeriod': 12*ONE_HOUR,
        }

        self.default_config = {
            'defaultTime': 0,
            'expirationFee': self.expiration_fee,
            'liquidityPercent': 0,
            'maxAllowedMeasureLag': ONE_HOUR*4,  # 4 hours
            'maxMeasurePeriod': ONE_DAY*31,  # 31 day
            'maxPeriodToBetsClose': ONE_DAY*31,  # 31 day
            'measureStartFee': self.measure_start_fee,
            'minMeasurePeriod': 60*5,  # 5 min
            'minPeriodToBetsClose': 60*5,  # 5 min
            'minPoolSize': 0,
            'oracleAddress': self.oracle_address,
            'rewardCallFee': 100_000,
        }

        self.init_storage = {
            'events': {},
            'betsFor': {},
            'betsAgainst': {},
            'providedLiquidityFor': {},
            'providedLiquidityAgainst': {},
            'liquidityShares': {},
            'depositedBets': {},
            'lastEventId': 0,
            'closeCallId': None,
            'measurementStartCallId': None,
            'newEventConfig': self.default_config,
            'manager': self.manager,

            'liquidityPrecision': 1_000_000,
            'ratioPrecision': 100_000_000,
            'sharePrecision': 100_000_000,
            'targetDynamicsPrecision': 1_000_000,
        }

        # Originating contract:
        client = self.client.using(key='bootstrap1')
        client.reveal()

        contract = ContractInterface.from_file(contract_fn)

        opg = contract.using(shell=self.get_node_url(), key='bootstrap1')
        opg = opg.originate(initial_storage=self.init_storage)
        opg = opg.fill().sign().inject()

        self.bake_block()

        # Find originated contract address by operation hash
        opg = client.shell.blocks['head':].find_operation(opg['hash'])
        op_result = opg['contents'][0]['metadata']['operation_result']
        contract_address = op_result['originated_contracts'][0]

        # Load originated contract from blockchain
        self.contract = client.contract(contract_address)
        self.contract = self.contract.using(
            shell=self.get_node_url(),
            key='bootstrap1')

        # Perform real contract call
        call = self.contract.newEvent(self.default_event_params).with_amount(300_000)
        opg = call.inject()

        self.bake_block()

        # Get injected operation and convert to ContractCallResult
        opg = client.shell.blocks['head':].find_operation(opg['hash'])
        result = ContractCallResult.from_operation_group(opg)[0]

        # It is hard to work with time, did not found solution yet,
        #     can't change time with bake_block operation
        # It is needed to originate oracle, and to provide current prices
        #     to make proper tests


In [15]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_contract_interactions (__main__.ContractInteractionsTestCase) ... /home/fortune/Documents/DeFiProjects/FortuneCrystalBall/pytezos-env/lib/python3.9/site-packages/ply/lex.py:319: ResourceWarning: unclosed <socket.socket fd=57, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.1', 56180), raddr=('172.17.0.6', 8732)>
  for lexre, lexindexfunc in self.lexre:
/home/fortune/Documents/DeFiProjects/FortuneCrystalBall/pytezos-env/lib/python3.9/site-packages/ply/lex.py:319: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.1', 56190), raddr=('172.17.0.6', 8732)>
  for lexre, lexindexfunc in self.lexre:
/home/fortune/Documents/DeFiProjects/FortuneCrystalBall/pytezos-env/lib/python3.9/site-packages/simplejson/decoder.py:400: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.1', 56206), raddr=('1